In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
import torch_tensorrt

import numpy as np
import gzip
import pickle
import os
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import gc
import time
import random
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, accuracy_score, cohen_kappa_score
from collections import deque
import pandas as pd
import math


import sys
sys.path.append('..')
from slp_package.slp_functions import create_merged_game_data_df
from slp_package.input_dataset import InputDataSet
import slp_package.pytorch_functions as slp_pytorch_functions

def set_seed(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if you are using CUDA
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)
torch.cuda.is_available()

True

In [2]:
def asses_model(model_name, y_pred, y_test, labels_order):
    print()
    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    kappa = cohen_kappa_score(y_pred, y_test)

    # Print accuracy and Cohen Kappa score with explanations
    print(f'Accuracy of {model_name}: {accuracy:.4f}')
    print(f'Cohen Kappa Score of {model_name}: {kappa:.4f}')

    # Calculate the normalized predicted label count
    unique_pred, counts_pred = np.unique(y_pred, return_counts=True)
    unique_test, counts_test = np.unique(y_test, return_counts=True)
    normalized_counts_pred = {k: v / counts_test[np.where(unique_test == k)[0][0]] for k, v in zip(unique_pred, counts_pred)}
    
    # Calculate the percent the model over or under predicted the labels using the specified label order
    sorted_values = [normalized_counts_pred[k] - 1 if k in normalized_counts_pred else 0 for k in labels_order]

    # Plotting the percent the model over or under predicted the labels
    plt.figure(figsize=(2*len(labels_order)+10, 4))
    plt.bar(labels_order, sorted_values, color=['green' if x > 0 else 'blue' for x in sorted_values])
    plt.title(f'Percent Model {model_name} Over or Under Predicted Labels')
    plt.xlabel('Labels')
    plt.ylabel('Percent Over/Under Prediction')
    
    # Center y-axis and set equal extension above and below
    max_extent = max(abs(min(sorted_values)), abs(max(sorted_values))) * 1.05
    plt.ylim(-max_extent, max_extent)
    plt.axhline(y=0, color='gray', linewidth=0.8)
    plt.show()

    # Display each confusion matrix on its own row
    for norm in [None, 'true', 'pred']:
        plt.figure(figsize=(len(labels_order)+4, len(labels_order)+4))
        ax = plt.gca()
        ConfusionMatrixDisplay.from_predictions(
            y_test, y_pred, normalize=norm, ax=ax,
            xticks_rotation='vertical', labels=labels_order
        )
        ax.title.set_text(f'{model_name} Confusion Matrix ({"Not Normalized" if norm is None else "Normalized by " + norm})')
        plt.tight_layout()
        plt.show()
    

In [3]:
source_data = ['ranked','public','mango']

general_features = {
    'stage_name': ['FOUNTAIN_OF_DREAMS','FINAL_DESTINATION','BATTLEFIELD','YOSHIS_STORY','POKEMON_STADIUM','DREAMLAND'],
    'num_players': [2],
    'conclusive': [True],
}
player_features = {
    # 'netplay_code': ['MANG#0'],
    # 'character_name': ['FALCO'],
    'character_name': ['PIKACHU', 'PICHU'],
    'character_name': ['DR_MARIO', 'MARIO'],
    # 'character_name': ['FOX', 'FALCO', 'MARTH', 'CAPTAIN_FALCON', 'SHEIK'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'MARIO', 'PEACH', 'ROY', 'LUIGI', 'YOUNG_LINK', 'DONKEY_KONG', 'PICHU', 'KIRBY'],
    # 'character_name': ['FOX', 'CAPTAIN_FALCON', 'SHEIK', 'FALCO', 'GAME_AND_WATCH', 'MARTH', 'LINK', 'ICE_CLIMBERS', 'SAMUS', 'GANONDORF', 'BOWSER', 'MEWTWO', 'YOSHI', 'PIKACHU', 'JIGGLYPUFF', 'NESS', 'DR_MARIO', 'PEACH', 'LUIGI', 'DONKEY_KONG'],
    'type_name': ['HUMAN']
    
}
opposing_player_features = {
    # 'character_name': ['MARTH'],
    # 'netplay_code': ['KOD#0', 'ZAIN#0']
    'type_name': ['HUMAN']
}
label_info = {
    'source': ['player'], # Can be 'general', 'player
    # 'feature': ['netplay_code']
    'feature': ['character_name']
}


In [4]:
dataset = InputDataSet(source_data, general_features, player_features, opposing_player_features, label_info)

print(dataset.dataset['labels'].value_counts())
dataset.dataset.head()

DR_MARIO    4202
MARIO       1713
Name: labels, dtype: int64


/workspace/slp_jaspar/random_conv_resnet/../slp_package/input_dataset.py:95: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  processed_df = pd.concat([player_1_df, player_2_df], ignore_index=True)


,stage_name,num_players,conclusive,player_character_name,player_type_name,opposing_player_type_name,player_inputs_np_sub_path,length,labels
0,YOSHIS_STORY,2,True,DR_MARIO,HUMAN,HUMAN,mango\DR_MARIO\14760f74-d6d0-479f-8439-fdf5adc...,10660,DR_MARIO
1,BATTLEFIELD,2,True,DR_MARIO,HUMAN,HUMAN,mango\DR_MARIO\c9f01df5-e378-4135-941d-e3d1689...,8127,DR_MARIO
2,BATTLEFIELD,2,True,DR_MARIO,HUMAN,HUMAN,mango\DR_MARIO\66bfa1d1-f744-4745-84b0-e796c67...,11120,DR_MARIO
3,FOUNTAIN_OF_DREAMS,2,True,DR_MARIO,HUMAN,HUMAN,mango\DR_MARIO\9d97358f-afc9-4074-a8d8-2658dab...,6988,DR_MARIO
4,POKEMON_STADIUM,2,True,DR_MARIO,HUMAN,HUMAN,mango\DR_MARIO\55113736-8401-46a1-b1dc-a3258f1...,9511,DR_MARIO


In [5]:
labels_order =  dataset.number_of_segments_per_game(3600, 8000)
print(labels_order)
labels_order = labels_order['Label'].values

      Label  Count  Shift
0  DR_MARIO   4177   3807
1     MARIO   1710   1633


In [6]:
train_df, test_df  = dataset.train_test_split_dataframes(test_ratio = .20, val = False)
print(train_df.shape)
print(test_df.shape)

                           player_inputs_np_sub_path  length  num_segments  \
0  mango\DR_MARIO\80bce95c-b480-4eed-b282-184f207...    7400             1   
1  ranked\DR_MARIO\016036ac-5f30-445b-846e-a06b8b...   11205             2   
2  ranked\DR_MARIO\2010efeb-a9f0-4c22-8c57-a0fd7c...   15012             3   
3  public\DR_MARIO\7687db8c-99ff-4d8e-881c-714cce...    7397             1   
4  public\DR_MARIO\0dc06526-b116-4d79-90ce-0bcab6...    7391             1   

     labels  encoded_labels  
0  DR_MARIO               0  
1  DR_MARIO               0  
2  DR_MARIO               0  
3  DR_MARIO               0  
4  DR_MARIO               0  
(12800, 6)
(3200, 6)


In [7]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        # self.segment_index = df['segment_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        # Start and end of the segment
        segment_start = self.segment_start_index[idx]
        segment_end = self.segment_start_index[idx] + self.segment_length[idx]
        
        segment = segment[:,int(segment_start):int(segment_end)]
        

        segment[-5] = (segment[-5] > .5)

        # Convert to PyTorch tensors
        segment_tensor = torch.from_numpy(segment).float()
        label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)
        return segment_tensor, label_tensor
    
def prepare_data_loaders(train_df, test_df, batch_size, num_workers):
    # Initialize datasets
    train_dataset = TrainingDataset(train_df,True)
    # val_dataset = TrainingDataset(file_paths_val, labels_val)
    test_dataset = TrainingDataset(test_df,True)

    # Initialize data loaders
    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True,persistent_workers=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True),
        # 'val': DataLoader(val_dataset, batch_size=2**9, num_workers=num_workers, shuffle=False, pin_memory=True,persistent_workers=True)
    }
    return loaders

In [8]:
def train_model(model, criterion, optimizer, loaders, device, random_conv, num_epochs=1):
    # Initialize progress bar for the epochs
    epoch_progress = tqdm(range(num_epochs), desc='Training Progress', unit='epoch')
    for epoch in epoch_progress:
        model.train()
        epoch_loss_sum = 0
        epoch_total = 0

        for _, input_cpu, target_cpu in enumerate(loaders['train']):
            # Move data to the appropriate device
            input_gpu = input_cpu.to(device)
            
            # Zero the parameter gradients
            optimizer.zero_grad()
            
            # Forward pass
            pred_gpu = model(random_conv(input_gpu))
            
            # Calculate loss
            loss = criterion(pred_gpu, target_cpu.to(device)) 
            loss.backward()
            
            # Clip gradients to avoid exploding gradients
            # clip_grad_norm_(model.parameters(), max_norm=1.0)
            
            # Perform a single optimization step
            optimizer.step()
            
            # Accumulate loss sum for accurate average calculation
            epoch_loss_sum += loss.item() * target_cpu.size(0)
            epoch_total += target_cpu.size(0)

        # Calculate average loss for the epoch
        epoch_loss = epoch_loss_sum / epoch_total

        # Update progress bar with the final loss of the epoch
        epoch_progress.set_postfix(Loss=f'{epoch_loss:.10f}')

    return model

def predict(model, loaders, loader, device):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        eval_loader_tqdm = tqdm(loaders[loader], unit='batch')
        
        for _, target_cpu in enumerate(eval_loader_tqdm):
            target_gpu = target_cpu.to(device)
            output_gpu = model(target_gpu)
            # output_gpu = torch.sigmoid(output_gpu)
            
            predictions.append(torch.sigmoid(output_gpu).cpu().numpy())
            targets.append(target_cpu.numpy())
    
    predictions = np.concatenate(predictions, axis=0)
    targets = np.concatenate(targets, axis=0)
    
    

    return predictions, targets

In [9]:
class RandomConv1d(nn.Module):
    def __init__(self):
        super(RandomConv1d, self).__init__()
        # Randomly set the number of input and output channels
        in_channels = 9  # Arbitrary range from 1 to 9
        out_channels = 9 # Arbitrary range from 1 to 9

        # Random kernel size, usually a small odd number is practical
        kernel_size = np.random.choice([3, 5, 7, 11, 13])



        # Random dilation, usually 1 or a small number
        dilation = np.random.randint(1,10)

        # Random padding, usually related to the kernel size
        padding = np.random.randint(0, kernel_size // 2 + 1)
        # Create the Conv1d layer with random parameters
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size,
                               padding=padding, dilation=dilation)

        # Randomly initialize weights and biases
        nn.init.uniform_(self.conv1.weight, -0.1, 0.1)
        if self.conv1.bias is not None:
            nn.init.uniform_(self.conv1.bias, -0.1, 0.1)

    def forward(self, x):
        x = self.conv1(x)
        return x

# Example of using the class
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# model = torch.compile(model,mode = 'max-autotune')

In [10]:
class TrainingDataset(Dataset):
    """
    Custom dataset for loading game segments from compressed numpy files.
    """
    def __init__(self, df, transform=None):
        self.file_paths = df['player_inputs_np_sub_path'].to_numpy()
        self.encoded_labels = df['encoded_labels'].to_numpy()
        self.segment_start_index = df['segment_start_index'].to_numpy()
        self.segment_length = df['segment_length'].to_numpy()
        self.transform = transform

    def __len__(self):
        """Returns the total number of samples in the dataset."""
        return len(self.file_paths)

    def __getitem__(self, idx):
        """Loads and returns a sample from the dataset at the specified index."""
        with gzip.open('/workspace/melee_project_data/input_np/' + self.file_paths[idx].replace('\\','/'), 'rb') as f:
            segment = np.load(f)
        
        segment_start = self.segment_start_index[idx]
        segment_end = segment_start + self.segment_length[idx]
        
        segment = segment[:, int(segment_start):int(segment_end)]
        segment[-5] = (segment[-5] > .5)

        segment_tensor = torch.from_numpy(segment).float()
        label_tensor = torch.tensor(self.encoded_labels[idx], dtype=torch.long)

        return segment_tensor, label_tensor

def prepare_data_loaders(train_df, test_df, batch_size, num_workers):
    train_dataset = TrainingDataset(train_df, True)
    test_dataset = TrainingDataset(test_df, True)

    loaders = {
        'train': DataLoader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True),
        'test': DataLoader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False, pin_memory=True, persistent_workers=True),
    }
    return loaders

def train_model(model, criterion, optimizer, loaders, device, conv_layer, num_epochs=1):
    epoch_progress = tqdm(range(num_epochs), desc='Training Progress', unit='epoch')
    for epoch in epoch_progress:
        model.train()
        epoch_loss_sum = 0
        epoch_total = 0

        for _, (input_cpu, target_cpu) in enumerate(loaders['train']):  # Correct unpacking
            input_gpu = conv_layer(input_cpu.to(device))  # Preprocess with random_conv
            target_gpu = target_cpu.to(device).float().unsqueeze(1)
            
            optimizer.zero_grad()
            output_gpu = model(input_gpu)
            # print(target_gpu)
            # print(output_gpu)
            loss = criterion(output_gpu, target_gpu)
            loss.backward()
            optimizer.step()
            
            epoch_loss_sum += loss.item() * target_cpu.size(0)
            epoch_total += target_cpu.size(0)

        epoch_loss = epoch_loss_sum / epoch_total
        epoch_progress.set_postfix(Loss=f'{epoch_loss:.10f}')

    return

def predict(model, loaders, loader, conv_layer, device):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        # eval_loader_tqdm = tqdm(loaders[loader], unit='batch')
        
        for _, (input_cpu, labels_cpu) in enumerate(loaders[loader]):
            target_gpu = conv_layer(input_cpu.to(device))
            output_gpu = model(target_gpu)
            # output_gpu = torch.sigmoid(output_gpu)
            
            predictions.append(torch.sigmoid(output_gpu).cpu().numpy())
            targets.append(labels_cpu.numpy())
    
    return  np.concatenate(predictions, axis=0), np.concatenate(targets, axis=0)
    # targets = np.concatenate(targets, axis=0)


In [11]:
from Encode_ResNet_Model import ResNet50

channels = 9
batch_size =  8
num_workers = 16
loaders = prepare_data_loaders(train_df, test_df, batch_size, num_workers)
criterion = nn.BCEWithLogitsLoss()
epochs = 10
# model = torch.compile(model,mode = 'max-autotune')





In [12]:
# model = ResNet50(2, channels).to('cuda')
# optimizer = Adam(model.parameters(), lr=0.0001)
# random_conv = RandomConv1d().to('cuda').eval()
# train_model(model, criterion, optimizer, loaders, 'cuda', random_conv , num_epochs=epochs)
# pred = predict(model, loaders, 'test', random_conv, 'cuda')

In [13]:
average_preds = np.zeros((3200,1))

In [14]:
for n in range(100):
    model = ResNet50(2, channels).to('cuda')
    optimizer = Adam(model.parameters(), lr=0.0001)
    # model = torch.compile(model)
    random_conv = RandomConv1d().to('cuda').eval()
    train_model(model, criterion, optimizer, loaders, 'cuda', random_conv , num_epochs=epochs)
    pred, target = predict(model, loaders, 'test', random_conv, 'cuda')
    average_preds = (average_preds * (n) + pred) / (n+1)
    
    pred_label = pred > .5
    average_pred_label = average_preds > .5
    kappa = cohen_kappa_score(pred_label, target)
    accuracy = accuracy_score(pred_label, target)
    print(f'Accuracy of {n}: {accuracy:.4f}')
    print(f'Cohen Kappa Score of {n}: {kappa:.4f}')
    
    kappa = cohen_kappa_score(average_pred_label, target)
    accuracy = accuracy_score(average_pred_label, target)
    print(f'Running Accuracy {n}: {accuracy:.4f}')
    print(f'Running Cohen Kappa Score {n}: {kappa:.4f}')
    print()

Training Progress: 100%|██████████| 10/10 [04:23<00:00, 26.30s/epoch, Loss=0.1098772552]


Accuracy of 0: 0.8678
Cohen Kappa Score of 0: 0.7356
Running Accuracy 0: 0.8678
Running Cohen Kappa Score 0: 0.7356



Training Progress: 100%|██████████| 10/10 [04:53<00:00, 29.32s/epoch, Loss=0.0745252047]


Accuracy of 1: 0.8778
Cohen Kappa Score of 1: 0.7556
Running Accuracy 1: 0.9306
Running Cohen Kappa Score 1: 0.8612



Training Progress: 100%|██████████| 10/10 [04:37<00:00, 27.77s/epoch, Loss=0.0854723208]


...

Training Progress: 100%|██████████| 10/10 [04:46<00:00, 28.65s/epoch, Loss=0.1348568210]


Accuracy of 33: 0.8087
Cohen Kappa Score of 33: 0.6175
Running Accuracy 33: 0.9513
Running Cohen Kappa Score 33: 0.9025



Training Progress: 100%|██████████| 10/10 [04:34<00:00, 27.43s/epoch, Loss=0.1200689620]


Accuracy of 34: 0.8556
Cohen Kappa Score of 34: 0.7112
Running Accuracy 34: 0.9513
Running Cohen Kappa Score 34: 0.9025



Training Progress:  80%|████████  | 8/10 [03:44<00:55, 27.99s/epoch, Loss=0.0903039929]

In [ ]:
    # pred_label = pred > .5
    average_pred_label = average_preds > .5
    # kappa = cohen_kappa_score(pred_label, target)
    # accuracy = accuracy_score(pred_label, target)
    # print(f'Accuracy of {n}: {accuracy:.4f}')
    # print(f'Cohen Kappa Score of {n}: {kappa:.4f}')
    
    kappa = cohen_kappa_score(average_pred_label, target)
    accuracy = accuracy_score(average_pred_label, target)
    print(f'Running Accuracy {n}: {accuracy:.4f}')
    print(f'Running Cohen Kappa Score {n}: {kappa:.4f}')


Running Accuracy 281: 0.9300
Running Cohen Kappa Score 281: 0.8600
